In [17]:
import re
import pandas as pd
from seqeval.scheme import IOB2
from seqeval.metrics import classification_report as cr
from seqeval.metrics import performance_measure
from seqeval.scheme import IOB2
from seqeval.metrics import performance_measure
import conllu
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score, precision_score, recall_score, classification_report
from torch.utils.data import Dataset
import torch
import numpy as np
import os




with open(r"C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\Results\NCRFpp\prediction_NSCP_samples.txt", "r") as f:
    preds_norne = f.readlines()


In [18]:
preds_norne = [re.sub('\n', '', line) for line in preds_norne]



In [19]:
# Heavily insipired from:
# https://github.com/ltgoslo/NorBERT/blob/main/benchmarking/experiments/dataset.py

class CoNLLDataset(Dataset):
    def __init__(self, x_tokens, y_labels, ner_vocab=None):
        self.tokens = [[x for x in entry] for entry in x_tokens]
        self.ner_labels = [[y for y in entry] for entry in y_labels]

        # hard coded ner_vocab to avoid random shuffled instanciation of ordering of ner_vocab
        self.ner_vocab = ner_vocab
        self.ner_indexer = {i: n for n, i in enumerate(self.ner_vocab)}
    
    def __getitem__(self, index):
        tokens = self.tokens[index]
        ner_labels = self.ner_labels[index]

        x = tokens
        y = torch.LongTensor([self.ner_indexer[i] if i in self.ner_vocab
                              else self.ner_indexer['@UNK'] for i in ner_labels])
        return x, y

    def __len__(self):
        return len(self.tokens)



In [20]:
ner_vocab =     ['B-GPE_LOC', 'I-DRV', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 
                'I-GPE_ORG', 'B-GPE_ORG', 'B-EVT', 'B-DRV', 'I-PROD', 'B-ORG', 'B-MISC',
                'I-MISC', 'I-GPE_LOC', 'B-LOC', 'I-ORG', 'I-EVT', 'O', '@UNK']


In [21]:
"""LOADING NPSC SAMPLES """


import csv

with open(r'C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\IOB NPSC sample\200_annoterte_setninger.csv', newline='', encoding="UTF-8") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    left_col = []
    right_col = []
    for row in csvreader:
        left_col.append(row[0])
        right_col.append(row[1])

x, y = [], []

with open(r"C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\IOB NPSC sample\parl_annotation_comma_fixed 100 sents.csv", "r", encoding="UTF-8") as f_input:
    reader = csv.reader(f_input, delimiter=",")
    for i, row in enumerate(reader):
        if i%2:
            y.append(row)
        else:
            x.append(row)


def create_sublists(lst):
    sublists = []
    sublist = []
    for element in lst:
        if element == '':
            if sublist:
                sublists.append(sublist)
                sublist = []
        else:
            sublist.append(element)

    if sublist:
        sublists.append(sublist)

    return sublists

y200 = create_sublists(right_col)
x200 = create_sublists(left_col)
x += x200
y += y200



def lower_list_of_lists(lists):
    return [[word.lower() for word in sublist] for sublist in lists]


test=lower_list_of_lists(x)
test_dataset = CoNLLDataset(test, y, ner_vocab)

""" Gold labels for test_dataset"""
gold_labels = []
for sentence_labels in test_dataset.ner_labels:
    for label in sentence_labels:
        gold_labels.append(label)


In [22]:
gold_labels = []
for sentence_labels in test_dataset.ner_labels:
    for label in sentence_labels:
        gold_labels.append(label)

In [23]:
len(gold_labels)

5438

In [24]:
"""Making list of lists preds, same len as NorNE gold labels"""
""" test_dataset_ner_labels should be test_dataset.ner_labels """

def split_list_preds(preds, test_dataset_ner_labels):
    split_list_preds = []
    start = 0

    for sublist in test_dataset_ner_labels:
        end = start + len(sublist)
        split_list_preds.append(preds[start:end])
        start = end
    return split_list_preds

In [25]:
def transform_to_binary(set):
    binary_labels = []
    for label in set:
        if label == "O":
            binary_labels.append(0)
        else:
            binary_labels.append(1)
    return binary_labels

In [26]:
def gold_labels_unlabled_data(unlabeled):
    length = 0
    gold_labels = []
    for sentences in unlabeled:
        label = []
        length += len(sentences)
        for token in sentences:
            if token[0].isupper() == True: # Checks if token has captial letter
                label.append(1)
                
            else:
                label.append(0) # Adds 0 if token has lower 
        gold_labels.append(label)
    
    return gold_labels

In [27]:
""" Reults section """

' Reults section '

In [28]:
norne_preds_binary = transform_to_binary(preds_norne)

gold_binary = transform_to_binary(gold_labels)




In [29]:
preds_split_sentences = split_list_preds(preds_norne, test_dataset.ner_labels)

In [30]:
# CLASSIFICATION REPORT
norne_cr_df_all_labels = classification_report(gold_labels, preds_norne, labels = ner_vocab[:-2], digits=5, output_dict=True)
norne_cr_IOB2 = cr(test_dataset.ner_labels, preds_split_sentences, mode='strict', scheme=IOB2, digits=4, output_dict=True)
norne_cr_binary = classification_report(gold_binary, norne_preds_binary, digits=5, output_dict=True)

norne_cr_df_all_labels = pd.DataFrame(norne_cr_df_all_labels).transpose()
norne_cr_df_IOB2 = pd.DataFrame(norne_cr_IOB2).transpose()
norne_cr_df_binary = pd.DataFrame(norne_cr_binary).transpose()


## CONFUSION MATRIX
norne_cf_matrix_no_O = confusion_matrix(gold_labels, preds_norne, labels = ner_vocab[:-2])
norne_cf_matrix_O = confusion_matrix(gold_labels, preds_norne, labels = ner_vocab[:-1])
norne_cr_matrix_binary = confusion_matrix(gold_binary, norne_preds_binary)

norne_cf_df_no_O = pd.DataFrame(norne_cf_matrix_no_O)
norne_cf_df_O = pd.DataFrame(norne_cf_matrix_O)
norne_cf_df_binary = pd.DataFrame(norne_cr_matrix_binary)

norne_cf_df_no_O.columns = ner_vocab[:-2]
norne_cf_df_no_O.index = ner_vocab[:-2]
norne_cf_df_O.columns = ner_vocab[:-1]
norne_cf_df_O.index = ner_vocab[:-1]


# class report
norne_cr_df_all_labels.to_csv("NPSC_SAMPLES_cr_df_all_labels", mode="w")
norne_cr_df_IOB2.to_csv("NPSC_SAMPLES_cr_df_IOB2", mode="w")
norne_cr_df_binary.to_csv("norne_cr_df_binary", mode="w")

#confusion matrix
norne_cf_df_no_O.to_csv("NPSC_SAMPLES_cf_df_no_O", mode="w")
norne_cf_df_O.to_csv("NPSC_SAMPLES_cf_df_O", mode="w")
norne_cf_df_binary.to_csv("NPSC_SAMPLES_cf_df_binary", mode="w")

c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [31]:
# parl_cr = classification_report(gold_labels_parl, y_pred_parl_binary, digits=5, output_dict=True)
# parl_cf_matrix = confusion_matrix(gold_labels_parl, y_pred_parl_binary)

# parl_cr_df = pd.DataFrame(parl_cr).transpose()
# parl_cf_df = pd.DataFrame(parl_cf_matrix)

# parl_cf_df.columns = [0, 1]
# parl_cf_df.index = [0, 1]

# parl_cr_df = pd.DataFrame(parl_cr).transpose()
# parl_cf_df = pd.DataFrame(parl_cf_matrix)

# parl_cf_df.columns = [0, 1]
# parl_cf_df.index = [0, 1]

# parl_cr_df.to_csv("parl_classification_report_ncrf", mode="w")
# parl_cf_df.to_csv("parl_matrix_ncrf", mode="w")